# **ERA5 Synoptic Map for SWEX IOP #10**
## This notebook performs the following tasks:
> - #### Makes a synoptic meteorology plot over the West Coast/East Pacific Ocean using ERA5 reanalysis data. We average 24 hourly ERA5 files and plot that average for the period covering SWEX IOP #10 (May 12-13, 2022).

## **Import packages**
#### Links to documentation for packages
> - #### [pathlib](https://docs.python.org/3/library/pathlib.html) | [pygrib](https://jswhit.github.io/pygrib/) | [numpy](https://numpy.org/doc/1.21/) | [cartopy](https://scitools.org.uk/cartopy/docs/latest/) | [matplotlib](https://matplotlib.org/3.5.3/index.html) |  
> - #### Documentation for packages linked above should mostly correspond to the most stable versions, which may not be the exact versions used when creating this notebook.
> - #### Comments are also included in the actual code cells. Some comments contain links that point to places where I copied or adapted code to fit my needs. Although I tried to these include links for all instancs of copying, it is possible that there may code snippets that I did not do this for.

In [ ]:
#-----------------------------------------------------
#Entire packages
import pathlib
import pygrib
import numpy as np

#cartopy imports
import cartopy.crs as ccrs

#matplotlib imports
import matplotlib.patheffects
import matplotlib.font_manager
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import colormaps
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

#function notebook
%run ./functions_swex_iop_10.ipynb
#-----------------------------------------------------

## **Define variables that point to paths for relevant data**
### Data Information
> - #### ERA5 Reanalysis: [Hourly Data on Single Levels (1959-Present)](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview)
>> - #### Variables available in downloaded single level files (order matters): Mean sea level pressure, u wind at 10m, v wind at 10m
>> - #### Filename convention: era5_single_level_YYYYMMDDHH_utc.grib; Example: era5_single_level_2022051308_utc.grib; Time zone of filenames is UTC.
> - #### ERA5 Reanalysis: [Hourly Data on Pressure Levels (1959-Present)](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=overview)
>> - #### Variables available in downloaded pressure level files (order matters): Geopotential, u wind component, v wind component, temperature, specific humidity, and relative humidity at specific pressure level indicated by file name. 
>> - #### Filename convention: era5_pressure_level_PPP_YYMMDDHH_utc; Example of 500hPa file: era5_pressure_level_500_2022051308_utc.grib; PPP=Pressure level; Time zone of filenames is UTC.

In [ ]:
#-----------------------------------------------------
#Define a glob of paths for each ERA5 dataset (i.e. single and pressure level files)

#Define a start and end date to grab ERA5 files for
#Format (YYYYMMDDHH)
start_date_files = 2022051217 
end_date_files   = 2022051317

#Grab files we want based on glob pattern and list comprehension
#ChatGPT came up with this solution for me.
glob_era5_single_level_files       = [file for date in range(start_date_files, end_date_files) for file in sorted(pathlib.Path("./SWEX2022_datasets/ERA5/single_level/").glob(f"*{date}*"))]
glob_era5_pressure_level_850_files = [file for date in range(start_date_files, end_date_files) for file in sorted(pathlib.Path("./SWEX2022_datasets/ERA5/pressure_level_850/").glob(f"*{date}*"))]
glob_era5_pressure_level_500_files = [file for date in range(start_date_files, end_date_files) for file in sorted(pathlib.Path("./SWEX2022_datasets/ERA5/pressure_level_500/").glob(f"*{date}*"))]

#Display first few glob paths to ensure we got the files we want
display(glob_era5_single_level_files[0])
display(glob_era5_pressure_level_850_files[0])
display(glob_era5_pressure_level_500_files[0])
#-----------------------------------------------------

## **Make a plot that shows the synoptic conditions for the time period covering SWEX IOP #10**

### Notes
> - #### The next two cells do the work to create synoptic plot for SWEX IOP #10 (May 12-13, 2022)
> - #### The first cell computes the average of 24 hourly ERA5 surface, 850hPa, and 500 hPa files starting at 10:00 PDT (17:00UTC; inclusive) on 2022-05-12 and ending a 10:00 PDT (17:00 UTC; exclusive) on 2022-05-13
> - #### The second cell plots the following variables on a cartopy map:
>> - #### Mean Sea Level Pressure (contour lines) 
>> - #### 850 hPa winds (wind vectors)
>> - #### 500 hPa geopotential heights (filled contour; derived quantity from ERA5 geopotential values)
>> - #### Check out the inline comments for additional details about specific lines of code

In [ ]:
#-----------------------------------------------------
#ERA5 GRIB FILE PROCESSING
#-----------------------------------------------------
#Define cartopy plot domain in lat/lon coordinates
lon_lat_tick_num = [4, 4]
lon_lat_extent = [-150, -110, 20, 60]
lon_lat_ticks  = [-150, -110, 20, 60]

#Define lon/lat extents for cropping our ERA5 data
#I usually crop data a bit bigger than the plotting domain, just to account for any continuity issues for specific variables
#Note: Because we are over the US west coast, and thus using negative longitudes, we subtract degrees on the left longitude and add degrees to the right longitude to crop our data beyond our map
left_lon_crop  = lon_lat_extent[0]-0.25
right_lon_crop = lon_lat_extent[1]+0.25
lower_lat_crop = lon_lat_extent[2]-0.25
upper_lat_crop = lon_lat_extent[3]+0.25

#Create lists that will store the daily averaged ERA5 variables
daily_list_single_level_mslp         = []
daily_list_pressure_level_850_u_wind = []
daily_list_pressure_level_850_v_wind = []
daily_list_pressure_level_500_geo    = []
#-----------------------------------------------------
#For every file in the list of daily files do the following:
for file_index, (era5_single_level_file, era5_pressure_level_850_file, era5_pressure_level_500_file) in enumerate(zip(glob_era5_single_level_files, glob_era5_pressure_level_850_files, glob_era5_pressure_level_500_files)):

    #Open grib files for each type of ERA5 file
    grbs_single_level       = pygrib.open(str(era5_single_level_file))
    grbs_pressure_level_850 = pygrib.open(str(era5_pressure_level_850_file))
    grbs_pressure_level_500 = pygrib.open(str(era5_pressure_level_500_file))

    #SINGLE LEVEL FILES

    #Select each parameter for analysis & plotting (single level file)
    grb_single_level_mslp = grbs_single_level.select()[0] #mean sea level pressure (unit = Pa)

    #Grab lat/lon/values for each single level parameter (area was cropped during download of data)
    data_single_level_mslp, lats_single_level_mslp, lons_single_level_mslp = grb_single_level_mslp.data(lat1=lower_lat_crop, lat2=upper_lat_crop, lon1=left_lon_crop, lon2=right_lon_crop)

    #Append data to daily composite list
    daily_list_single_level_mslp.append(data_single_level_mslp)

    #850 hPa FILES

    #Select each parameter for analysis & plotting (pressure level file)
    grb_pressure_level_850_u_wind = grbs_pressure_level_850.select()[1] #u wind at pressure level (unit = m/s)
    grb_pressure_level_850_v_wind = grbs_pressure_level_850.select()[2] #v wind at pressure level (unit = m/s)

    #Grab lat/lon/values for each pressure level parameter (area was cropped during download of data)
    data_pressure_level_850_u_wind, lats_pressure_level_850_u_wind, lons_pressure_level_850_u_wind = grb_pressure_level_850_u_wind.data(lat1=lower_lat_crop, lat2=upper_lat_crop, lon1=left_lon_crop, lon2=right_lon_crop)
    data_pressure_level_850_v_wind, lats_pressure_level_850_v_wind, lons_pressure_level_850_v_wind = grb_pressure_level_850_v_wind.data(lat1=lower_lat_crop, lat2=upper_lat_crop, lon1=left_lon_crop, lon2=right_lon_crop)

    #Append data to daily composite list
    daily_list_pressure_level_850_u_wind.append(data_pressure_level_850_u_wind)
    daily_list_pressure_level_850_v_wind.append(data_pressure_level_850_v_wind)

    #500 hPa FILES

    #Select each parameter for analysis & plotting (pressure level file)
    grb_pressure_level_500_geo = grbs_pressure_level_500.select()[0] #geopotential at pressure level (unit = m^2 s^-2)

    #Grab lat/lon/values for each pressure level parameter (area was cropped during download of data)
    data_pressure_level_500_geo, lats_pressure_level_500_geo, lons_pressure_level_500_geo = grb_pressure_level_500_geo.data(lat1=lower_lat_crop, lat2=upper_lat_crop, lon1=left_lon_crop, lon2=right_lon_crop)

    #Append data to daily composite list
    daily_list_pressure_level_500_geo.append(data_pressure_level_500_geo)
#-----------------------------------------------------
#Once we have looped through a days worth of files, take mean of each list of arrays that contain a days worth of ERA5 files
#https://stackoverflow.com/questions/40173006/numpy-mean-and-std-over-every-terms-of-arrays
daily_composite_single_level_mslp         = np.mean(daily_list_single_level_mslp, axis=0)
daily_composite_pressure_level_850_u_wind = np.mean(daily_list_pressure_level_850_u_wind, axis=0)
daily_composite_pressure_level_850_v_wind = np.mean(daily_list_pressure_level_850_v_wind, axis=0)
daily_composite_pressure_level_500_geo    = np.mean(daily_list_pressure_level_500_geo, axis=0)
#-----------------------------------------------------

In [ ]:
#-----------------------------------------------------
#Cartopy Plotting
#-----------------------------------------------------
#Define map and data coordinate reference system for our cartopy map
plot_crs = ccrs.PlateCarree()
data_crs = ccrs.PlateCarree()

#Define font properties for different items
fontdict_title_labels    = {'fontsize': 24, 'fontweight': 'bold', 'fontname': 'Nimbus Roman'}
fontdict_text_color_bar  = {'fontsize': 24, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
fontdict_text_annotation = {'fontsize': 24, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
fontdict_tick_labels     = {'fontsize': 24, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
kwargs_clabels           = {'fontsize': 24, 'colors':'grey', 'fmt':'%1.00f', 'zorder':10}
fontdict_quiver_key      = {'size': 24, 'weight': 'bold', 'family': 'Nimbus Roman'}

#Run cartopy plot helper function
fig, ax = cartopy_basemap_subplots(plot_crs=plot_crs, data_crs=data_crs, fig_size=(20,10), 
                                   nrows=1, ncols=1, wspace_float=0, hspace_float=0,
                                   lon_lat_extent=lon_lat_extent, lon_lat_ticks=lon_lat_ticks, lon_lat_tick_num=[5,5], 
                                   lon_lat_ticks_on=True, xtick_ytick_set_list=[True],
                                   high_res_coastline=False, 
                                   high_res_wrf_topo_sb_bool=False, 
                                   low_res_wrf_topo_sb_bool=False, 
                                   low_res_wrf_topo_ca_bool=False, 
                                   wrf_topo_colorbar_each_plot_bool=False, 
                                   wrf_topo_colorbar_entire_figure_bool=False,
                                   scale_bar_bool=False, scale_bar_position=None, scale_bar_length=None, 
                                   inset_ca_bool=False, inset_bbox_position=None, 
                                   ocean_color=None)
#-----------------------------------------------------
#Plot MSLP with contour lines (ERA5 MSLP is given in units of Pa. We divide by 100 to get units into hPa)

#Define contour line levels
contour_line_levels = np.arange(900,1100,2)

#Plot contour lines
contour_line_plot   = ax.contour(lons_single_level_mslp, 
                                 lats_single_level_mslp, 
                                 daily_composite_single_level_mslp/100, 
                                 levels=contour_line_levels, colors='silver', linewidths=2, linestyles='solid', transform=data_crs)

#Add path effects to contour lines
contour_line_plot.set(path_effects=[matplotlib.patheffects.Stroke(linewidth=3, foreground='black'), matplotlib.patheffects.Normal()])

#Plot contour line labels
contour_line_labels = ax.clabel(contour_line_plot, contour_line_plot.levels, inline=True, manual=False, inline_spacing=7, **kwargs_clabels)

#Make the contour labels bold after creating them
#Recommended by ChatGPT
for label in contour_line_labels:
    label.set_path_effects([matplotlib.patheffects.withStroke(linewidth=4, foreground='white'), matplotlib.patheffects.Normal()])
#-----------------------------------------------------
#Plot wind vectors for every n-th grid point
nth_pt = 4

#Plot vectors for 850hPa winds
quiver = ax.quiver(lons_pressure_level_850_u_wind[::nth_pt,::nth_pt], 
                   lats_pressure_level_850_u_wind[::nth_pt,::nth_pt], 
                   daily_composite_pressure_level_850_u_wind[::nth_pt,::nth_pt], 
                   daily_composite_pressure_level_850_v_wind[::nth_pt,::nth_pt], 
                   pivot='middle', zorder=2, transform=data_crs)
#-----------------------------------------------------
#Define colorbar levels and colorbar norms for plotting
#https://stackoverflow.com/questions/48613920/use-of-extend-in-a-pcolormesh-plot-with-discrete-colorbar
#https://mycarta.wordpress.com/2012/10/14/the-rainbow-is-deadlong-live-the-rainbow-part-4-cie-lab-heated-body/
#https://stackoverflow.com/questions/72215114/matplotlib-colorbar-extend-in-different-color

#Define contourf levels
contourf_level_min  = 535
contourf_level_max  = 595
contourf_level_step = 5
contourf_levels = np.arange(contourf_level_min, contourf_level_max+contourf_level_step, contourf_level_step)

#Define cmap for contourf and colorbar and set out of bound values
contourf_cmap = colormaps.get_cmap('plasma').copy()
contourf_cmap.set_extremes(under='black', over='white', bad='None')

#Set the norm for contourf
contourf_norm = mcolors.BoundaryNorm(contourf_levels, ncolors=contourf_cmap.N, clip=False)

#Plotting geopotential height with contourf
#The ERA5 data is geopotential only; To convert to geopotential height we divide the geopotential values by the gravitational acceleration constant (9.80655)
#We then divide by 10 to convert from height values in meters (m) to decameters (dam)
#https://confluence.ecmwf.int/display/CKB/ERA5%3A+compute+pressure+and+geopotential+on+model+levels%2C+geopotential+height+and+geometric+height#:~:text=To%20obtain%20the%20geopotential%20height,s2%20in%20the%20IFS.
contourf_plot = ax.contourf(lons_pressure_level_500_geo, 
                            lats_pressure_level_500_geo, 
                            (daily_composite_pressure_level_500_geo/9.80655)/10, 
                            levels=contourf_levels, cmap=contourf_cmap, norm=contourf_norm, transform=data_crs)

#Colorbar for contourf
#https://matplotlib.org/3.1.1/gallery/axes_grid1/demo_colorbar_with_axes_divider.html
#Second Answer: https://stackoverflow.com/questions/30030328/correct-placement-of-colorbar-relative-to-geo-axes-cartopy
divider = make_axes_locatable(ax)
cax     = divider.append_axes('right', size='2%', pad=0.25, axes_class=plt.Axes)
cbar    = plt.colorbar(contourf_plot, cax=cax, orientation='vertical', spacing='uniform', drawedges=True, ticks=contourf_levels[::1])
cbar.set_label('Geopotential Height (dam)', color='black', labelpad=20, **fontdict_text_color_bar)

#Set font for colorbar tick lables
#https://stackoverflow.com/questions/7257372/set-font-properties-to-tick-labels-with-matplot-lib/7280803
ticks_font = matplotlib.font_manager.FontProperties(family='Nimbus Roman', style='normal', size=24, weight='normal', stretch='normal')
for label in cbar.ax.get_yticklabels():
    label.set_fontproperties(ticks_font)
#-----------------------------------------------------
#Add in a symbol to show the location of Santa Barbara area in each plot
ax.scatter(-119.700209, 34.393321, s=500,  marker='*', color='white', edgecolor='black', transform=data_crs, zorder=5)

#Add title
ax.set_title(f'a) 12-13 May 2022 ERA5 Daily Average\nMSLP, 850-hPa Winds, 500-hPa Heights', **fontdict_title_labels)

#Define inset axis for quiver key
ax1 = inset_axes(ax, width='100%',height='100%',loc='upper right', borderpad=0, bbox_to_anchor=(0.80, 0.95, 0.20, 0.05), bbox_transform=ax.transAxes)

#Remove ticks from inset axis
ax1.tick_params(labelleft=False,labelbottom=False,left=False,bottom=False)

#Add quiver key to inset axis
ax1.quiverkey(quiver, X=0.83, Y=0.97, U=10, label=r'10 m s$^{-1}$', labelpos='E', fontproperties=fontdict_quiver_key)
#-----------------------------------------------------  
#Save figure with tight layout
#https://stackoverflow.com/questions/11837979/removing-white-space-around-a-saved-image-in-matplotlib
plt.savefig('./figures/figure_02a_era5_synoptic_map.png', bbox_inches='tight', dpi=500)

#Show figure
plt.show()
#-----------------------------------------------------